<a href="https://colab.research.google.com/github/Ejeat12/Profolio/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Gradient calculations with AutoGrad

In [ ]:
# Gradient calculations of x,y,z
x = torch.randn([3], requires_grad= True)
y = x + 2
z = y*y*2
# calculate dz/dx
z.backward(x)
print(y)



tensor([3.0522, 1.6226, 2.6621], grad_fn=<AddBackward0>)


In [ ]:
# Prevent gradient histroy
with torch.no_grad():
  y = x + 2
y



tensor([3.0522, 1.6226, 2.6621])

# BackPropagation

In [ ]:
# Inputs
x = torch.tensor(1.0)
y = torch.tensor(2.0)

# Parameters
w = torch.tensor(1.0, requires_grad= True)

# Forward pass and compute loss
y_hat = x * w
loss = (y_hat - y)**2

# Backward pass
loss.backward()
w.grad

tensor(-2.)

# Gradient Descent with Autograd and Backpropagation

In [ ]:
# initalize inputs and weights
x = torch.tensor([1,2,3,4], dtype = torch.float32)
y = torch.tensor([2,4,6,8], dtype = torch.float32)
w = torch.tensor(0.0, requires_grad= True)

# Model prediction
def forward(x):
  return w * x

# Loss
def loss(y, y_pred):
  return((y- y_pred)**2).mean()


# Hyperparameters
learning_rate = 1e-2
epochs = 100

# Train the model
for epoch in range(epochs):
  
  
  # Prediction = forward pass
  y_hat = forward(x)
    
    
  #loss
  l = loss(y, y_hat)
    
    
  # calculate gradients
  l.backward()
    
  
  # update weights with gradient descent
  with torch.no_grad():
    
   w-= learning_rate * w.grad
  
  
  # empty weights
  w.grad.zero_()
  
 
  
  # Get info about training steps
  if epoch % 10 == 0:
    print(f"epoch: {epoch+1}, loss: {l:.8f}")
    print(f"prediction after training: {forward(5):.3f}")
 
 


epoch: 1, loss: 30.00000000
prediction after training: 1.500
epoch: 11, loss: 1.16278565
prediction after training: 8.327
epoch: 21, loss: 0.04506890
prediction after training: 9.671
epoch: 31, loss: 0.00174685
prediction after training: 9.935
epoch: 41, loss: 0.00006770
prediction after training: 9.987
epoch: 51, loss: 0.00000262
prediction after training: 9.997
epoch: 61, loss: 0.00000010
prediction after training: 10.000
epoch: 71, loss: 0.00000000
prediction after training: 10.000
epoch: 81, loss: 0.00000000
prediction after training: 10.000
epoch: 91, loss: 0.00000000
prediction after training: 10.000


# Training Pipeline: Model, Loss, and Optimizer

In [ ]:
from torch.optim import optimizer
from torch.nn.modules.linear import Linear


import torch
import torch.nn as nn
# 0) Training samples, watch the shape!
X = torch.tensor([[1],[2],[3],[4]], dtype = torch.float32)
y = torch.tensor([[2],[4],[6],[8]], dtype = torch.float32)



# 0) create a test sample
x_test = torch.tensor([5], dtype = torch.float32)


# 1) Design input and ouput shape of model
n_samples, n_features = X.shape
input_size = n_features
output_size = n_features

# Build a custom model class
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        return self.lin(x)
model = LinearRegression(input_size, output_size)


# 2) Define hyperparameters
learning_rate = 1e-2
epochs = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


# 3) Training loop
for epoch in range(epochs):

  

    # predict = forward pass with our model
    y_pred = model(X)

    # loss
    l = loss(y, y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()
    # Get inforamtion
    if epoch % 10 == 0:
      print(f"epoch: {epoch+1}, loss: {l:.8f}")
print(f'Prediction after training: f(5) = {model(x_test).item():.3f}')     

epoch: 1, loss: 31.08114052
epoch: 11, loss: 0.80963933
epoch: 21, loss: 0.02612339
epoch: 31, loss: 0.00555058
epoch: 41, loss: 0.00473456
epoch: 51, loss: 0.00444625
epoch: 61, loss: 0.00418711
epoch: 71, loss: 0.00394340
epoch: 81, loss: 0.00371388
epoch: 91, loss: 0.00349770
Prediction after training: f(5) = 10.098


# Logistic Regression

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# 0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [ ]:
# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)
  
# 2) Hyperparameters
learning_rate = 1e-2
epochs = 100
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 3) Training loop
for epoch in range(epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    l = loss(y_pred, y_train)

    # Backward pass and update
    l.backward()
    optimizer.step()
    

    # zero grad before new step
    optimizer.zero_grad()
    if epoch % 10 == 0:
      print(f"epoch: {epoch+1}, loss: {l.item():.8f}")


# step 5: get accuracy
with torch.no_grad():

  y_pred = model(X_test)
  y_pred_class = y_pred.round()
  accuracy = y_pred_class.eq(y_test).sum() / float(y_test.shape[0])
  print(f"accuracy: {accuracy.item():.4f}")

epoch: 1, loss: 1.17586195
epoch: 11, loss: 0.81495833
epoch: 21, loss: 0.60845131
epoch: 31, loss: 0.49141195
epoch: 41, loss: 0.41879171
epoch: 51, loss: 0.36955634
epoch: 61, loss: 0.33387169
epoch: 71, loss: 0.30669358
epoch: 81, loss: 0.28520012
epoch: 91, loss: 0.26769727
accuracy: 0.9211


# Softmax and Cross Entropy in PyTorch

# Level 1:Train a model(basic)

In [ ]:
!pip install pytorch_lightning -qq
!pip install --upgrade wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
# libraries
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import pytorch_lightning as pl


In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)



In [ ]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
#Define the training set
dataset = MNIST(os.getcwd(), download = True, transform = transforms.ToTensor())
train_loader = DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:00<00:00, 430176633.19it/s]

Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38590536.42it/s]


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 165479659.19it/s]


Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6519688.15it/s]

Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



In [ ]:

#  define model
autoencoder = LitAutoEncoder(Encoder(), Decoder())




In [ ]:
# Train without training loop
optimizer = autoencoder.configure_optimizers()

for batch_idx, batch in enumerate(train_loader):
  loss = autoencoder.training_step(batch, batch_idx)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()



# Level 2:ADD A VALIDATION AND TEST SET

In [ ]:
import torch.utils.data as data
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
# Load datasets
transform = transforms.ToTensor()
train_data = datasets.MNIST(root = "MNIST", download = True, train = True, transform = transform)
test_data = datasets.MNIST(root = "MNIST", download = True, train = True, transform = transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 99608786.13it/s]


Extracting MNIST/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 94049451.73it/s]


Extracting MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24102053.36it/s]


Extracting MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22438785.36it/s]


Extracting MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/MNIST/raw



In [ ]:
# Add a test loop
class LitAutoEncoder(pl.LightningModule):
  def training_step(self, batch, batch_idx):
    pass

  def test_step(self, batch, batch_idx):
    x, y = batch
    x = x.view(x.size(0), -1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    test_loss = F.mse_loss(x_hat, x)
    self.log("test loss", test_loss)

     

In [ ]:
# train the test loop
from torch.utils.data import DataLoader
from transformers import Trainer

# initlaizae the trainer
trainer = Trainer()
trainer.test(model, dataloaders = DataLoader(test_data))



RuntimeError: ignored

In [ ]:
# train both data and validation split
trainer = Trainer()
train_loader = DataLoader(train_data)
val_loader = DataLoader(val_data)
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Save a checkpoint to a specfic path at every epoch end
trainer = Trainer(default_root_dir = "some/path/")




In [ ]:
# Load a lightning module based on checkpoint
model = LitAutoEncoder.load_from_checkpoint("some/path/")
#disable dropout
model.eval()
# make prediction with x
y_pred = model(x)

In [ ]:
# save hyperparameters
class LitAutoEncoder(pl.lightningModule):
  def _init_(self, learning_rate):
    super()._init_()
    self.save_hyperparameters()